In [4]:
%%writefile seminar.cu

#include <stdio.h>
#include <stdlib.h>
#include <opencv2/opencv.hpp>
#include <iostream>

using namespace cv;
using namespace std;

// kernel za Laplaceov 7x7 filter
__global__ void laplacianFilter(uchar3 *input, uchar3 *output, int rows, int cols) {
    // x i y pozicije piksela
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < cols && y < rows) {  // provjera granica slike
        int idx = y * cols + x;  // indeks trenutnog piksela
        float sumR = 0.0, sumG = 0.0, sumB = 0.0;

        // računanje Laplaceovog operatora za svaki kanal u 7x7 susjedstvu
        for (int dx = -3; dx <= 3; ++dx) {
            for (int dy = -3; dy <= 3; ++dy) {
                int nx = min(max(x + dx, 0), cols - 1); // krajnji se zrcale
                int ny = min(max(y + dy, 0), rows - 1);
                int neighborIdx = ny * cols + nx;  // indeks susjednog piksela
                sumR += input[neighborIdx].x;
                sumG += input[neighborIdx].y;
                sumB += input[neighborIdx].z;
              }
          }
            sumR = 48 * input[idx].x - sumR;
            sumG = 48 * input[idx].y - sumG;
            sumB = 48 * input[idx].z - sumB;

            // ograničavanje vrijednosti do 255 i spremanje u izlaz
            output[idx].x = (sumR < 0) ? 0 : ((sumR > 255) ? 255 : sumR);
            output[idx].y = (sumG < 0) ? 0 : ((sumG > 255) ? 255 : sumG);
            output[idx].z = (sumB < 0) ? 0 : ((sumB > 255) ? 255 : sumB);
    }
}

int main(int argc, char** argv) {
    if (argc != 2) {
        printf("usage: DisplayImage.out <Image_Path>\n");
        return -1;
    }

    // Učitavanje slike
    Mat image;
    image = imread(argv[1], IMREAD_COLOR);
    if (image.empty()) {  // provjera uspješnog učitavanja slike
        printf("Nije moguće otvoriti ili pronaći sliku.\n");
        return -1;
    }
    int height = image.rows;
    int width = image.cols;

    // ispis informacija o slici
    size_t sizeInBytes = image.total() * image.elemSize();
    int sizeMat = static_cast<int>(sizeInBytes);
    printf("Veličina slike: %i bajtova\n", sizeMat);
    printf("Dimenzije slike: %i x %i\n", height, width);

    // Spremanje originalne slike
    printf("Spremanje originalne slike...\n");
    imwrite("split_copy.jpg", image);

    // alokacija memorije na uređaju za sliku
    uchar3 *d_input, *d_output;
    uchar3 *a = image.ptr<uchar3>(0);
    cudaMalloc((void**)&d_input, sizeMat);
    cudaMalloc((void**)&d_output, sizeMat);

    // kopiranje ulazne slike na uređaj
    cudaMemcpy(d_input, a, sizeMat, cudaMemcpyHostToDevice);

    // definiranje dimenzija bloka i grid-a
    dim3 blockSize(32, 32);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // poziv kernel fje
    laplacianFilter<<<gridSize, blockSize>>>(d_input, d_output, height, width);
    cudaDeviceSynchronize();

    // kopiranje filtrirane slike na host
    uchar3 *b = (uchar3 *)malloc(sizeMat);
    cudaMemcpy(b, d_output, sizeMat, cudaMemcpyDeviceToHost);
    printf("Kopiranje filtriranih podataka s uređaja.\n");

    // stvaranje izlazne slike i spremanje
    Mat image_out = Mat(height, width, CV_8UC3, (unsigned char*)b);
    imwrite("new.jpg", image_out);
    printf("Filtrirana slika spremljena kao new.jpg\n");

    cudaFree(d_input);
    cudaFree(d_output);
    free(b);

    return 0;
}


Overwriting seminar.cu


In [5]:
!nvcc seminar.cu -o seminar.out `pkg-config --cflags --libs opencv4` -w
!./seminar.out slika.jpg

Veličina slike: 3108105 bajtova
Dimenzije slike: 819 x 1265
Spremanje originalne slike...
Kopiranje filtriranih podataka s uređaja.
Filtrirana slika spremljena kao new.jpg


In [ ]:
import cv2
print(cv2.__version__)

4.8.0
